In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
import random


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer


In [ ]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('dataset.json').read())

words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))


In [ ]:
training_sentences = []
training_labels = []
label_encoder = LabelEncoder()

for doc in documents:
    bag = [0]*len(words)
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    for w in words:
        if w in pattern_words:
            bag[words.index(w)] = 1

    training_sentences.append(bag)
    training_labels.append(classes.index(doc[1]))

train_x = np.array(training_sentences)
train_y = np.array(training_labels)


In [ ]:
from tensorflow.keras.utils import to_categorical
train_y = to_categorical(training_labels)


In [ ]:
model = Sequential()
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Eğitim ve test verilerini ayırma
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# Modeli yeniden eğitme
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Modeli test etme ve doğruluk oranını hesaplama
predictions = model.predict(test_x)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_y, axis=1)

accuracy = accuracy_score(true_classes, predicted_classes)
print(f"Modelin doğruluk oranı: {accuracy * 100:.2f}%")



Epoch 1/200
3/3 [==============================] - 5s 9ms/step - loss: 1.5758 - accuracy: 0.3333
Epoch 2/200
3/3 [==============================] - 0s 12ms/step - loss: 1.5902 - accuracy: 0.1667
Epoch 3/200
3/3 [==============================] - 0s 10ms/step - loss: 1.7094 - accuracy: 0.2500
Epoch 4/200
3/3 [==============================] - 0s 8ms/step - loss: 1.6169 - accuracy: 0.2500
Epoch 5/200
3/3 [==============================] - 0s 10ms/step - loss: 1.5701 - accuracy: 0.1667
Epoch 6/200
3/3 [==============================] - 0s 13ms/step - loss: 1.5170 - accuracy: 0.2500
Epoch 7/200
3/3 [==============================] - 0s 18ms/step - loss: 1.5145 - accuracy: 0.4167
Epoch 8/200
3/3 [==============================] - 0s 7ms/step - loss: 1.5191 - accuracy: 0.2500
Epoch 9/200
3/3 [==============================] - 0s 12ms/step - loss: 1.4291 - accuracy: 0.5000
Epoch 10/200
3/3 [==============================] - 0s 13ms/step - loss: 1.6033 - accuracy: 0.1667
Epoch 11/200
3/3 [====

In [ ]:
def chatbot_response(text):
    bag = bag_of_words(text, words)
    result = model.predict(np.array([bag]))[0]
    result_index = np.argmax(result)
    tag = classes[result_index]

    if result[result_index] > 0.50:  # Güven eşiği
        for i in intents['intents']:
            if i['tag'] == tag:
                response = random.choice(i['responses'])
                break
    else:
        response = "Anlamadım, lütfen başka bir şey sorun."
    return response


In [ ]:
model.save('chatbot_modelv2.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def bag_of_words(text, words):
    # Kelimeleri tokenize et
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(text)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)


In [1]:
print("Chatbot'a hoş geldiniz! (çıkmak için 'çıkış' yazın)")
while True:
    inp = input("Sen: ")
    if inp.lower() == "çıkış":
        print("Chatbot ile görüşmeyi sonlandırdınız. Görüşmek üzere!")
        break
    response = chatbot_response(inp)
    print("Chatbot: ", response)


Chatbot'a hoş geldiniz! (çıkmak için 'çıkış' yazın)
Sen: selam


NameError: name 'chatbot_response' is not defined